In [10]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle
import datetime

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [11]:
# Driver setting
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
headers = {
    "User-Agent": user_agent
}

def setWebdriver():
    service = ChromeService(executable_path=ChromeDriverManager().install())    # 크롬 드라이버 최신 버전 설정

    options = ChromeOptions()
    options.add_argument('user-agent=' + user_agent)
    # options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    # options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    #options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    #options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    #options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(service=service, options=options)

    return driver

In [27]:
# Scroll down
def scrollDown(driver, whileSeconds): 
    end = driver.execute_script("return document.body.scrollHeight")
    # while (True):
    #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #     time.sleep(rd.uniform(0.8, 1.2))
    #     new_end = driver.execute_script("return document.body.scrollHeight")
    #     if new_end == end:
    #         break
    #     end = new_end

    start = datetime.datetime.now() # 스크롤 다운 시작 시간 설정
    end = start + datetime.timedelta(seconds=whileSeconds) # 스크롤 다운 종료 시간 설정
    with tqdm(total=whileSeconds, desc='Scrolling Down...', leave='False') as pbar:
        while (datetime.datetime.now() < end):
            # 페이지 맨 아래로 스크롤 다운
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            time.sleep(1)        
            pbar.update(1)

    return

In [13]:
# make URL
def makeURL():
    global query, start_date, end_date
    # 검색어, 시작 날짜, 종료 날짜 입력
    query = input("검색어를 입력하세요: ") # 검색어 입력, keyword: 우크라이나
    start_date = re.sub(r'[^0-9]', '', input("시작 날짜를 yyyy.mm.dd 형식으로 입력하세요: ")) # 시작 날짜(정규식을 이용하여 숫자만 추출)
    end_date = re.sub(r'[^0-9]', '', input("종료 날짜를 yyyy.mm.dd 형식으로 입력하세요: "))    # 종료 날짜(정규식을 이용하여 숫자만 추출)
    # URL 설정
    url = f'https://search.naver.com/search.naver?ssc=tab.cafe.all&cafe_where=&query={query}&ie=utf8&st=rel&date_option=8&date_from={start_date}&date_to={end_date}&srchby=text&dup_remove=1&cafe_url=&without_cafe_url=&sm=tab_opt&nso=so%3Add%2Cp%3Afrom20230101to20240404&nso_open=1&prdtype=0'

    return url

In [14]:
# get title&URL
def getTitleURL(driver):
    # 블로그 글 element 수집
    try:
        articles = driver.find_elements(By.CLASS_NAME, "detail_box");   #print(articles)
    except:
        print(f'articles error...')
    # title_list, url_list 초기화
    title_list = [] # 제목 리스트
    url_list = []   # URL 리스트
    # title, url 수집
    for article in tqdm(articles, desc="Colleting title and URL..."):    
        try:
            # title
            title = article.find_element(By.CLASS_NAME, "title_link").text;      title_list.append(title);       #print(f'Title: {title}')
            # url
            url = article.find_element(By.CLASS_NAME, "title_link").get_attribute("href");
            url_list.append(url);           #print(f'URL: {url}')
        except:
            print(f'title or url error...')
    
    # print('-'*200)

    return title_list, url_list

In [29]:
# blog contents crawling
def getContents(driver, title_list, url_list):
    mainText_list = [] # 블로그 본문 리스트 초기화
    commentCnt_list = [] # 댓글 수 리스트 초기화
    comment_list = []   # 댓글 리스트 초기화
    imgCnt_list = []    # 이미지 수 리스트 초기화     
    img_list = []    # 이미지 리스트 초기화
    videoCnt_list = []  # 동영상 수 리스트 초기화
    video_list = [] # 동영상 리스트 초기화

    for i, url in enumerate(tqdm(url_list, desc="Contents...")):
        driver.get(url)
        time.sleep(1) # 로딩 대기
        
        # print(f'Title: {title_list[i]}');   print(f'URL: {url}')

        iframes = driver.find_elements(By.TAG_NAME, "iframe")    # iframe 수집
        if len(iframes) > 0:    # iframe이 있는지 확인
            # for iframe in iframes:
            #     print(f'iframe name: {iframe.get_attribute("name")}')  # iframe 이름 출력
            driver.switch_to.frame('cafe_main') # iframe 요소로 전환

            # 블로그 본문 수집
            try:
                main_text = driver.find_element(By.CLASS_NAME, "se-main-container").text    # 블로그 본문 텍스트 수집, se-main-container, ContentRenderer
                mainText_list.append(main_text)    # 블로그 본문 리스트에 추가
                # print(f'Main text collected!');  # print(f'Main text: {main_text}'); 
            except:
                mainText_list.append(None) # 블로그 본문이 없는 경우 None 추가
                
            # 댓글 수 수집
            try:
                comment_cnt = int(driver.find_element(By.CLASS_NAME, "num").text)    # 댓글 수 수집
                commentCnt_list.append(comment_cnt) # 댓글 수 리스트에 추가
                # print(f'Comment count: {comment_cnt}')
            except:
                print(f'comment count error..., URL: {url}')
                driver.quit()

            # 댓글 수집
            if (comment_cnt > 0):
                try:
                    comments = [comment.text for comment in driver.find_elements(By.CLASS_NAME, "text_comment")]    # 댓글 수집
                    comment_list.append(comments)   # 댓글 리스트에 추가
                    # print(f'comments({len(comments)}): {comments}')
                except:
                    driver.quit()
                    print(f'comments error2..., URL: {url}')
            else:
                comment_list.append(None)   # 댓글이 없는 경우 None 추가

            # 이미지 수집
            try:
                images = [img.get_attribute('src') for img in driver.find_elements(By.TAG_NAME, "img")]    # 이미지 수집
                imgCnt_list.append(len(images)) # 이미지 수 리스트에 추가
                if (len(images) > 0):
                    img_list.append(images); 
                    # print(f'Images({len(images)}: {images})')
                else:
                    img_list.append(None);        
                    # print(f'There is no images...')
            except:
                print(f'images error..., URL: {url}')

            # 동영상 수집
            try:
                videos = [video.get_attribute('src') for video in driver.find_elements(By.CSS_SELECTOR, "iframe[src]")]    # 동영상 수집
                videoCnt_list.append(len(videos))   # 동영상 수 리스트에 추가
                if (len(videos) > 0): 
                    video_list.append(videos)
                    # print(f'Videos({len(videos)}: {videos})')
                else: 
                    video_list.append(None)
                    # print(f'There is no videos...')
            except:
                print(f'videos error..., URL: {url}')
        else:
            print(f'There is no iframe!!, URL: {url}')
            

        # print('-'*200)
        time.sleep(rd.uniform(0.1, 0.3))

    return mainText_list, commentCnt_list, comment_list, imgCnt_list, img_list, videoCnt_list, video_list

In [32]:
# 웹드라이버 초기화
driver = setWebdriver()

# URL 생성
url = makeURL()

# URL 접속
driver.get(url)

# 스크롤 다운
scrollDown(driver, 9)

# 블로그 title, url 수집
title_list, url_list = getTitleURL(driver)

# 블로그 콘텐츠 수집
mainText_list, commentCnt_list, comment_list, imgCnt_list, img_list, videoCnt_list, video_list = getContents(driver, title_list, url_list)

# shutdown webdriver 202
driver.quit()

# DataFrame 생성
df = pd.DataFrame({
    "title": title_list,
    "url": url_list,
    "main_text": mainText_list,
    "comment_cnt": commentCnt_list,
    "comment": comment_list,
    "img_cnt": imgCnt_list,
    "img": img_list,
    "video_cnt": videoCnt_list,
    "video": video_list
})
print(df)

# DataFrame 저장
file_path = "../data/";    file_name = f"naverCafe_crawling({query}, {start_date}-{end_date}).pkl"
df.to_pickle(file_path + file_name)

Contents...: 100%|██████████| 300/300 [08:40<00:00,  1.74s/it]


                                                 title  \
0                                          우크라이나는 기우는가   
1               [2312-5] 외국우표 분양 - 우크라이나, 폴란드 등 (97품목)   
2                                  우크라이나에 3조 49억 지원하네요   
3                              러시아-우크라이나를 잊으신 것은 아니겠죠?   
4                             러시아 우크라이나 전쟁에 대한 미국의 전략은   
..                                                 ...   
295                       우크라이나 전쟁은 결과적으론 이유를 뭐라 해야하나요   
296          우크라이나 지원) 이렇게 지원해주면 푸틴의 할아버지라도 아무 말 못함!!!   
297                                  우크라이나 전역에 공습경보 발령   
298  러시아가 도청으로 인한 우크라이나 지원사실은 넘어갈려고 했으나 윤석열 인터뷰 때문에...   
299     코오롱글로벌 주식 주가 전망 사우디 네옴시티 우크라이나 재건 수처리 인조잔디 관련주   

                                                   url  \
0    https://cafe.naver.com/wotat/1163281?art=ZXh0Z...   
1    https://cafe.naver.com/philatelyst/368733?art=...   
2    https://cafe.naver.com/momsofsongdoifez/120043...   
3    https://cafe.naver.com/vietnamsketch/444502?ar...   
4    https://

In [31]:
df

,title,url,main_text,comment_cnt,comment,img_cnt,img,video_cnt,video
0,우크라이나는 기우는가,https://cafe.naver.com/wotat/1163281?art=ZXh0Z...,2년 동안 이어진 전쟁\n전쟁 중에 굳이 하는 부정부패 척결\n우크라이나 동부 수복...,4,[문제는 러시아도 푸틀러만 웃고있지 나라는 점점 눈내리는 사우디가 되어가고 있는거 ...,12,[https://cafeptthumb-phinf.pstatic.net/MjAxNzA...,1,[https://www.youtube.com/embed/9kci2ylzkyw?fea...
1,"[2312-5] 외국우표 분양 - 우크라이나, 폴란드 등 (97품목)",https://cafe.naver.com/philatelyst/368733?art=...,분양양식\n▶ 분 양 자 격 : 만 19세이상(성인) 성실회원 이상만 가능합니다\n...,168,"[297, 307, 312, 313, 314, 334, 319.320.375, 3건...",212,[https://ssl.pstatic.net/static/cafe/cafe_pc/d...,0,None
2,우크라이나에 3조 49억 지원하네요,https://cafe.naver.com/momsofsongdoifez/120043...,"23억 달러 = 3조 49억 원\n내년에 우리나라 과학예산 2조 원 깎고\n교육부,...",34,[https://news.kbs.co.kr/news/mobile/view/view....,41,[https://ssl.pstatic.net/static/cafe/cafe_pc/d...,0,None
3,러시아-우크라이나를 잊으신 것은 아니겠죠?,https://cafe.naver.com/vietnamsketch/444502?ar...,러시아-우크라이나 전쟁에 대한 뉴스가 미얀마처럼 사라지고 있는 것은 아닌지하는 생각...,21,[현재 가장 무서운건 국제 사회 그리고 전세계인들이 전쟁에 둔감해 졌다는거\n\n얼...,30,[https://ssl.pstatic.net/static/cafe/cafe_pc/d...,3,[https://www.youtube.com/embed/-npwwX26Vjk?fea...
4,러시아 우크라이나 전쟁에 대한 미국의 전략은,https://cafe.naver.com/jaegebal/4912915?art=ZX...,어느 일방이 이기지 못하게 균형을 유지하는 것..\n그래서 제한적으로만 우크라이나 ...,4,[미국 돈떨어졌어유\n지원해줄 돈이 없음\n대출만기연장으로\n공화당하고 쇼부도 봐야...,11,[https://ssl.pstatic.net/static/cafe/cafe_pc/d...,0,None
...,...,...,...,...,...,...,...,...,...
295,우크라이나 전쟁은 결과적으론 이유를 뭐라 해야하나요,https://cafe.naver.com/appleiphone/7790775?art...,또 모의유엔 때문에 질문하네요...\n우크라이나랑 러시아가 예전에도 크림반도를 가지...,6,"[푸틴의 야욕, 나토 가입은 솔직히 표면상 이유라고 생각합니다\n유럽으로 가는 가스...",16,[https://cafeptthumb-phinf.pstatic.net/MjAyMjA...,0,None
296,우크라이나 지원) 이렇게 지원해주면 푸틴의 할아버지라도 아무 말 못함!!!,https://cafe.naver.com/fpsgame/3967222?art=ZXh...,우리군에서 널리 사용하고 있는 AMB!!!\n이것도 나름 군수물자 지원임.\n군용버...,8,"[ㅋㅋㅋㅋ, 짬처리 아님?, 오도봉고 지원해주면 좋을텐데, ㅋㅋㅋ, 포방부 연사 가...",27,[https://ssl.pstatic.net/static/cafe/cafe_pc/d...,0,None
297,우크라이나 전역에 공습경보 발령,https://cafe.naver.com/sbstvdocu/241246?art=ZX...,※ 수시간내 카스피해쪽에서 러시아의 대규모 미사일 공격이 있을 예정. 우크라이나에 ...,4,"[푸틴 시진핑 극페미 불법조선족댓긃루대들, 밭갈이들,북한공산주의 이슬람IS 이것들만...",18,[https://cafeptthumb-phinf.pstatic.net/MjAyNDA...,0,None
298,러시아가 도청으로 인한 우크라이나 지원사실은 넘어갈려고 했으나 윤석열 인터뷰 때문에...,https://cafe.naver.com/jaegebal/4504202?art=ZX...,며칠간 미국 도청 및 우크라이나 포탄 지원 관려 언론을 분석해보면...\n1. 러시...,65,"[빨갱이, 너는 러시아가 여전히 소련으로 밖에 생각안되지?\n러시아에 한류가 얼마나...",77,[https://cafeptthumb-phinf.pstatic.net/MjAyNDA...,0,None
